In [4]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.embeddings import HuggingFaceEmbeddings # HuggingFace Embeddings (Deprecation Warning 가능성 있음)
from langchain.llms import HuggingFacePipeline # <-- HuggingFacePipeline 임포트 (LangChain 핵심 또는 community에서)
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, pipeline # <-- pipeline 함수 임포트
from peft import PeftModel




# 1. CSV에서 `short_prompt`, `long_prompt` 로드
# 사용자 환경에 맞는 파일 경로로 수정해주세요.
df = pd.read_csv("/app/workspace/data/data9000_forRAG_test .csv")

# 2. E5 임베딩 모델 로드 및 `long_prompt` 벡터화
embedding_model_id = "intfloat/multilingual-e5-large" # E5 임베딩 모델 ID
# sentence_transformers 라이브러리로 모델 로드
embedder = SentenceTransformer(embedding_model_id)

# long_prompt 앞에 "passage:" 붙여서 벡터화 준비
passages = ["passage: " + p for p in df["prompt"].dropna()] # 결측치 제거 후 처리
# long_prompt가 비어있는 경우(NaN) 오류 방지
# embeddings = embedder.encode(passages, show_progress_bar=True, batch_size=32) # 오류 발생 시 passes->passages

# 3. Chroma 벡터 DB 저장
# long_prompt가 비어있는 행은 문서 생성에서 제외
# documents = [
#     Document(page_content=row["prompt"], metadata={"short_prompt": row["short_prompt"]})
#     for _, row in df.dropna(subset=["prompt"]).iterrows() # long_prompt에 결측치 없는 행만 사용
# ]

# Chroma에 긴 영어 프롬프트만 넣기
documents = [
    Document(page_content=row["prompt"]) # metadata 제외
    for _, row in df.dropna(subset=["prompt"]).iterrows()
]

# LangChain에서 HuggingFace 임베딩 래퍼 사용
# Deprecation Warning이 뜬다면 from langchain_huggingface.embeddings import HuggingFaceEmbeddings 사용 권장
embedding_wrapper = HuggingFaceEmbeddings(model_name=embedding_model_id)

# Chroma DB 구축 및 저장
# documents 리스트가 비어있지 않은 경우에만 DB 구축
if documents:
    chroma_db = Chroma.from_documents(documents, embedding_wrapper, persist_directory="./chroma_db")
    chroma_db.persist() # 디렉토리에 DB 파일 저장
    print("Chroma DB가 성공적으로 구축되었습니다.")
else:
    # 이미 DB가 저장되어 있다면 로드하여 사용하거나, 문서를 다시 확인해야 합니다.
    # 여기서는 문서가 없을 경우 메시지만 출력하고 Retriever는 None으로 둡니다.
    print("long_prompt 컬럼에 유효한 데이터가 없습니다. DB 구축을 건너뜁니다.")
    # 만약 저장된 DB를 로드하고 싶다면 아래 주석을 해제하고 경로를 확인하세요.
    # embedding_wrapper가 필요하므로 다시 정의해야 합니다.
    # embedding_wrapper = HuggingFaceEmbeddings(model_name=embedding_model_id)
    # try:
    #     chroma_db = Chroma(persist_directory="./chroma_db", embedding_function=embedding_wrapper)
    #     print("기존 Chroma DB를 로드했습니다.")
    # except Exception as e:
    #     print(f"기존 DB 로드 실패: {e}")
    #     chroma_db = None # 로드 실패 시 None

# 4. Chroma 벡터 DB에서 유사한 `long_prompt`를 검색할 수 있게 설정
retriever = None
if chroma_db is not None:
    retriever = chroma_db.as_retriever(search_kwargs={"k": 3}) # k=3으로 상위 3개 유사 프롬프트 검색
    print("Retriever 설정 완료.")
else:
    print("Chroma DB가 없어 Retriever를 설정할 수 없습니다.")


# 5. Kanana 모델 로드 (HuggingFacePipeline 사용 준비)
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0,
    llm_int8_skip_modules=None,
    llm_int8_enable_fp32_cpu_offload=True # 핵심!
)

# 사용자 환경에 맞는 모델/어댑터 경로로 수정해주세요.
adapter_path = "/home/piai/workspace/Kanana_Prompt5000/outputs"
base_model_id = "kakaocorp/kanana-nano-2.1b-instruct" # 베이스 모델 ID

print(f"Base 모델 로딩: {base_model_id}...")
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto",
    quantization_config=bnb_config
)
print("Base 모델 로딩 완료.")

print(f"LoRA 어댑터 로딩: {adapter_path}...")




/tmp/ipykernel_1763/3482250851.py:44: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding_wrapper = HuggingFaceEmbeddings(model_name=embedding_model_id)


RuntimeError: [91mYour system has an unsupported version of sqlite3. Chroma                     requires sqlite3 >= 3.35.0.[0m
[94mPlease visit                     https://docs.trychroma.com/troubleshooting#sqlite to learn how                     to upgrade.[0m

In [ ]:
# 6. LoRA 어댑터 붙이기
model = PeftModel.from_pretrained(base_model, adapter_path)
model.eval()
print("LoRA 어댑터 로딩 완료.")

print(f"Tokenizer 로딩: {adapter_path}...")
# 7. Tokenizer 불러오기
tokenizer = AutoTokenizer.from_pretrained(adapter_path, use_fast=False)
print("Tokenizer 로딩 완료.")


# 8. LangChain에서 사용할 Kanana 모델 설정 (HuggingFacePipeline 사용)

# Hugging Face Pipeline 객체 생성
# 로드한 model과 tokenizer를 pipeline에 연결하고 생성 파라미터 설정
print("HuggingFace Pipeline 설정 중...")
pipe = pipeline(
    "text-generation", # 수행할 작업 (텍스트 생성)
    model=model, # 로드된 PeftModel (LoRA 적용된 모델)
    tokenizer=tokenizer, # 로드된 Tokenizer
    max_new_tokens=300, # 생성될 최대 토큰 수 (generate 함수의 인자와 동일)
    max_length=500, # 파이프라인의 최대 길이 설정
    do_sample=False, # generate 함수의 인자와 동일
    repetition_penalty=1.5, # generate 함수의 인자와 동일
    eos_token_id=tokenizer.eos_token_id, # generate 함수의 인자와 동일
    pad_token_id=tokenizer.pad_token_id, # generate 함수의 인자와 동일
    num_return_sequences=1, # generate 함수의 인자와 동일
    no_repeat_ngram_size=2, # generate 함수의 인자와 동일
#     device=0 # 또는 "cuda". 모델이 이미 GPU에 로드되었다면 명시적으로 지정. CPU 사용 시 -1 또는 "cpu"
)
print("HuggingFace Pipeline 설정 완료.")

# HuggingFacePipeline 래퍼 생성
# 생성한 pipeline 객체를 HuggingFacePipeline 클래스에 전달
kanana_llm = HuggingFacePipeline(pipeline=pipe)
print("LangChain LLM 객체 (kanana_llm) 준비 완료.")

In [ ]:
# 9. 프롬프트 템플릿 구성 (검색된 `long_prompt` + 사용자 `short_prompt`)
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are a Midjourney prompt generator.
Refer to the following examples and rewrite the given simple idea into a detailed English prompt.

[Examples]
{context}

[User Input]
{question}

[Prompt]
"""
)
print("프롬프트 템플릿 설정 완료.")

# 10. RAG 체인 구성 (검색된 `long_prompt`와 사용자 `short_prompt` 결합)
rag_chain = None
if retriever is not None:
    print("RAG 체인 설정 중...")
    rag_chain = RetrievalQA.from_chain_type(
        llm=kanana_llm, # <-- 여기서 HuggingFacePipeline 객체를 사용합니다.
        retriever=retriever,
        chain_type_kwargs={"prompt": prompt_template}
    )
    print("RAG 체인 설정 완료.")
else:
    print("Retriever가 없어 RAG 체인을 설정할 수 없습니다.")




In [ ]:
# 11. 사용자 `short_prompt`로 Kanana에 전달하여 최종 long_prompt 생성
user_input = "하늘을 나는 고양이를 그려줘" # 예시 사용자 입력
if rag_chain is not None:
    print(f"\n사용자 입력: {user_input}")
    print("Kanana 모델로 long_prompt 생성 중...")
    response = rag_chain.run(user_input)

    # 12. 결과 출력
    print("\n--- 생성된 long_prompt ---")
    print(response)
else:
    print("\nRAG 체인이 설정되지 않아 long_prompt를 생성할 수 없습니다. Chroma DB 구축/로드 상태를 확인하세요.")